# DeepSeek API 输出解析 - OpenAI SDK

> 指导文章：[DeepSeek API 输出解析 - OpenAI SDK](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/DeepSeek%20API%20输出解析%20-%20OpenAI%20SDK.md)

从下方选择平台开始，替换 `your-api-key` 后点击 `►` 或使用 `Shift + 回车` 运行代码块。

在线链接：[Kaggle](https://www.kaggle.com/code/aidemos/deepseek-api-guide-2) | [Colab](https://colab.research.google.com/drive/1WT0jpeIzWewoN5cT12Uwi92d5_tNff2J?usp=sharing)

# 环境依赖

In [ ]:
!pip install openai

# DeepSeek

## 认识输出

### DeepSeek-Chat

In [1]:
from openai import OpenAI
import os

client = OpenAI(
    api_key="your-api-key", # 1：替换成对应的 API_Key，可以使用环境变量而非明文填写，即 api_key=os.getenv("DEEPSEEK_API_KEY")
    base_url="https://api.deepseek.com/v1", # 2：每个平台的 base_url 不同
)

# 单轮对话示例
completion = client.chat.completions.create(
    model="deepseek-chat", # 3：模型标识（model_id）可能存在差异
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant.'},
        {'role': 'user', 'content': '你是谁？'}
    ]
)

In [2]:
from pprint import pprint

pprint(completion.model_dump())

{'choices': [{'finish_reason': 'stop',
              'index': 0,
              'logprobs': None,
              'message': {'content': '您好！我是由中国的深度求索（DeepSeek）公司开发的智能助手DeepSeek-V3。如您有任何任何问题，我会尽我所能为您提供帮助。',
                          'function_call': None,
                          'refusal': None,
                          'role': 'assistant',
                          'tool_calls': None}}],
 'created': 1739067885,
 'id': '8e2cd3b1-c2cd-4bd6-9207-d843d8bc4535',
 'model': 'deepseek-chat',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': 'fp_3a5770e1b4',
 'usage': {'completion_tokens': 37,
           'prompt_cache_hit_tokens': 0,
           'prompt_cache_miss_tokens': 11,
           'prompt_tokens': 11,
           'prompt_tokens_details': {'cached_tokens': 0},
           'total_tokens': 48}}


#### 获取模型回复（choices）

In [3]:
print(completion.choices[0].message.content)

您好！我是由中国的深度求索（DeepSeek）公司开发的智能助手DeepSeek-V3。如您有任何任何问题，我会尽我所能为您提供帮助。


#### 获取用量信息（usage）

In [4]:
def print_chat_usage(completion):
    stats = completion.usage
    hit = stats.prompt_cache_hit_tokens
    miss = stats.prompt_cache_miss_tokens
    
    print(f"===== TOKEN 消耗明细 =====")
    print(f"输入: {stats.prompt_tokens} tokens [缓存命中: {hit} | 未命中: {miss}]")
    print(f"输出: {stats.completion_tokens} tokens")
    print(f"总消耗: {stats.total_tokens} tokens")
    
    # 按 DeepSeek 定价计算成本（单位：元）
    # - 输入: 2元/百万 Tokens（缓存命中 0.5元/百万 Tokens）
	# - 输出: 8元/百万 Tokens
    # 官方价格文档：https://api-docs.deepseek.com/zh-cn/quick_start/pricing/
    input_cost = (hit * 0.5 + miss * 2) / 1_000_000
    output_cost = stats.completion_tokens * 8 / 1_000_000
    total_cost = input_cost + output_cost
    
    print(f"\n===== 成本明细 =====")
    print(f"输入成本: ￥{input_cost:.4f} 元")
    print(f"输出成本: ￥{output_cost:.4f} 元")
    print(f"预估总成本: ￥{total_cost:.4f} 元")

print_chat_usage(completion)

===== TOKEN 消耗明细 =====
输入: 11 tokens [缓存命中: 0 | 未命中: 11]
输出: 37 tokens
总消耗: 48 tokens

===== 成本明细 =====
输入成本: ￥0.0000 元
输出成本: ￥0.0003 元
预估总成本: ￥0.0003 元


### DeepSeek-Reasoner

In [5]:
# 单轮对话示例
completion = client.chat.completions.create(
    model="deepseek-reasoner", # 3：换成推理模型
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant.'},
        {'role': 'user', 'content': '你是谁？'}
    ]
)

In [6]:
from pprint import pprint

pprint(completion.model_dump())

{'choices': [{'finish_reason': 'stop',
              'index': 0,
              'logprobs': None,
              'message': {'content': '您好！我是由中国的深度求索（DeepSeek）公司开发的智能助手DeepSeek-R1。如您有任何任何问题，我会尽我所能为您提供帮助。',
                          'function_call': None,
                          'reasoning_content': 'Alright, the user asked "你是谁？" '
                                               'which means "Who are you?" in '
                                               'Chinese. I need to respond in '
                                               'Chinese. Let me start by '
                                               'greeting them politely.\n'
                                               '\n'
                                               "I should mention that I'm an "
                                               'AI assistant created by '
                                               "DeepSeek. It's important to "
                                               'highlight my purpose, li

#### 获取模型回复（choices）

In [7]:
# 获取推理思考过程（Reasoner特有字段）
reasoning_content = completion.choices[0].message.reasoning_content
print(f"===== 模型推理过程 =====\n{reasoning_content}")

# 获取模型回复内容（与之前相同）
content = completion.choices[0].message.content
print(f"===== 模型回复 =====\n{content}")

===== 模型推理过程 =====
Alright, the user asked "你是谁？" which means "Who are you?" in Chinese. I need to respond in Chinese. Let me start by greeting them politely.

I should mention that I'm an AI assistant created by DeepSeek. It's important to highlight my purpose, like helping with information, answering questions, and offering advice. Keeping it friendly and open-ended would encourage further interaction. Let me make sure the tone is approachable and the information is clear.
===== 模型回复 =====
您好！我是由中国的深度求索（DeepSeek）公司开发的智能助手DeepSeek-R1。如您有任何任何问题，我会尽我所能为您提供帮助。


#### 获取用量信息（usage）

In [8]:
def print_reasoner_usage(completion):
    stats = completion.usage
    hit = stats.prompt_cache_hit_tokens
    miss = stats.prompt_cache_miss_tokens
    
    print(f"===== TOKEN 消耗明细 =====")
    print(f"输入: {stats.prompt_tokens} tokens [缓存命中: {hit} | 未命中: {miss}]")
    print(f"输出: {stats.completion_tokens} tokens")

    # 推理模型的token分解
    if details := stats.completion_tokens_details:
        reasoning = details['reasoning_tokens']
        final = stats.completion_tokens - reasoning
        print(f"├─ 推理过程: {reasoning} tokens")
        print(f"└─ 最终回答: {final} tokens")
    
    print(f"总消耗: {stats.total_tokens} tokens")
    
    # 按 DeepSeek 定价计算成本（单位：元）
    # - 输入Token: 4元/百万Tokens（未命中缓存 1元/百万Tokens）
	# - 输出Token: 16元/百万Tokens
    # 官方价格文档：https://api-docs.deepseek.com/zh-cn/quick_start/pricing/
    input_cost = (hit * 1 + miss * 4) / 1_000_000
    output_cost = stats.completion_tokens * 16 / 1_000_000
    total_cost = input_cost + output_cost
    
    print(f"\n===== 成本明细 =====")
    print(f"输入成本: ￥{input_cost:.4f} 元")
    print(f"输出成本: ￥{output_cost:.4f} 元")
    print(f"预估总成本: ￥{total_cost:.4f} 元")

print_reasoner_usage(completion)

===== TOKEN 消耗明细 =====
输入: 13 tokens [缓存命中: 0 | 未命中: 13]
输出: 134 tokens
├─ 推理过程: 95 tokens
└─ 最终回答: 39 tokens
总消耗: 147 tokens

===== 成本明细 =====
输入成本: ￥0.0001 元
输出成本: ￥0.0021 元
预估总成本: ￥0.0022 元


# 硅基流动

## 认识输出

### DeepSeek-Chat

In [9]:
from openai import OpenAI
import os

client = OpenAI(
    api_key="your-api-key", # 1：替换成对应的 API_Key，可以使用环境变量而非明文填写，即 api_key=os.getenv("SILICONFLOW_API_KEY")
    base_url="https://api.siliconflow.cn/v1", # 2：每个平台的 base_url 不同
)

# 单轮对话示例
completion = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-V3", # 3：模型标识（model_id）可能存在差异
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant.'},
        {'role': 'user', 'content': '你是谁？'}
    ]
)

In [10]:
from pprint import pprint

pprint(completion.model_dump())

{'choices': [{'finish_reason': 'stop',
              'index': 0,
              'logprobs': None,
              'message': {'content': '我是一个人工智能助手，由 OpenAI '
                                     '开发，旨在回答各种问题、提供信息和帮助解决任务。你可以问我任何问题，我会尽力提供有用的回答！有什么我可以帮助你的吗？',
                          'function_call': None,
                          'refusal': None,
                          'role': 'assistant',
                          'tool_calls': None}}],
 'created': 1739067903,
 'id': '0194e8864c5c970c36e16c8a98326619',
 'model': 'deepseek-ai/DeepSeek-V3',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': '',
 'usage': {'completion_tokens': 37, 'prompt_tokens': 11, 'total_tokens': 48}}


#### 获取模型回复（choices）

In [11]:
print(completion.choices[0].message.content)

我是一个人工智能助手，由 OpenAI 开发，旨在回答各种问题、提供信息和帮助解决任务。你可以问我任何问题，我会尽力提供有用的回答！有什么我可以帮助你的吗？


#### 获取用量信息（usage）

In [12]:
def print_chat_usage(completion, input_cost=2.0, output_cost=8.0, cache_hit_cost=0.5):
    """
    参数：
    - input_cost: 输入价格（元/百万 Tokens）
    - output_cost: 输出价格（元/百万 Tokens）
    - cache_hit_cost: 缓存命中价格（当平台不支持时自动退化到全价模式）

    按 DeepSeek 聊天模型定价设定默认成本（单位：元）：
    - 输入: 2元/百万 Tokens（缓存命中 0.5元/百万 Tokens）
	- 输出: 8元/百万 Tokens
    官方价格文档：https://api-docs.deepseek.com/zh-cn/quick_start/pricing/
    """
    stats = completion.usage

    # 尝试获取字段（兼容其他平台）
    hit = getattr(stats, 'prompt_cache_hit_tokens', 0)
    miss = getattr(stats, 'prompt_cache_miss_tokens', 
                  stats.prompt_tokens - hit if hasattr(stats, 'prompt_tokens') else 0)

    print(f"===== TOKEN 消耗明细 =====")
    # 仅在存在缓存机制时显示细节
    if hit + miss > 0:
        print(f"输入: {stats.prompt_tokens} tokens [缓存命中: {hit} | 未命中: {miss}]")
    else:
        print(f"输入: {stats.prompt_tokens} tokens")

    print(f"输出: {stats.completion_tokens} tokens")
    print(f"总消耗: {stats.total_tokens} tokens")

    # 动态成本计算
    input_cost = (hit * cache_hit_cost + miss * input_cost) / 1_000_000
    output_cost = stats.completion_tokens * output_cost / 1_000_000
    total_cost = input_cost + output_cost

    print(f"\n===== 成本明细 =====")
    print(f"输入成本: ￥{input_cost:.4f} 元")
    print(f"输出成本: ￥{output_cost:.4f} 元")
    print(f"预估总成本: ￥{total_cost:.4f} 元")

print_chat_usage(completion)

===== TOKEN 消耗明细 =====
输入: 11 tokens [缓存命中: 0 | 未命中: 11]
输出: 37 tokens
总消耗: 48 tokens

===== 成本明细 =====
输入成本: ￥0.0000 元
输出成本: ￥0.0003 元
预估总成本: ￥0.0003 元


### DeepSeek-Reasoner

In [13]:
# 单轮对话示例
completion = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1", # 3：换成推理模型
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant.'},
        {'role': 'user', 'content': '你是谁？'}
    ]
)

In [14]:
from pprint import pprint

pprint(completion.model_dump())

{'choices': [{'finish_reason': 'stop',
              'index': 0,
              'logprobs': None,
              'message': {'content': '您好！我是由深度求索（DeepSeek）公司开发的智能助手DeepSeek-R1。我可以协助您解答各类问题，提供信息查询、学习建议、科技资讯等支持。希望以专业又温暖的方式帮助到您！✨\n'
                                     '\n'
                                     '有具体需求随时告诉我哦～😊',
                          'function_call': None,
                          'reasoning_content': 'Good, I should start by '
                                               'greeting the user. Then '
                                               "explain that I'm an AI "
                                               'assistant created by DeepSeek '
                                               'to help with various tasks. '
                                               'Mention the areas I can assist '
                                               'with like answering questions, '
                                               'providing info, learning '
         

#### 获取模型回复（choices）

In [15]:
# 获取推理思考过程（Reasoner特有字段）
reasoning_content = completion.choices[0].message.reasoning_content
print(f"===== 模型推理过程 =====\n{reasoning_content}")

# 获取模型回复内容（与之前相同）
content = completion.choices[0].message.content
print(f"===== 模型回复 =====\n{content}")

===== 模型推理过程 =====
Good, I should start by greeting the user. Then explain that I'm an AI assistant created by DeepSeek to help with various tasks. Mention the areas I can assist with like answering questions, providing info, learning advice, and tech support. Use simple Chinese, friendly and professional tone. Keep it concise. Make sure to include the emojis and the structure as shown. Avoid markdown formatting.
===== 模型回复 =====
您好！我是由深度求索（DeepSeek）公司开发的智能助手DeepSeek-R1。我可以协助您解答各类问题，提供信息查询、学习建议、科技资讯等支持。希望以专业又温暖的方式帮助到您！✨

有具体需求随时告诉我哦～😊


#### 获取用量信息（usage）

In [16]:
def print_reasoner_usage(completion, input_cost=4.0, output_cost=16.0, cache_hit_cost=1.0):
    """
    参数：
    - input_cost: 输入价格（元/百万 Tokens）
    - output_cost: 输出价格（元/百万 Tokens）
    - cache_hit_cost: 缓存命中价格（当平台不支持时自动退化到全价模式）

    按 DeepSeek 推理模型定价设定默认成本（单位：元）：
    - 输入: 4元/百万 Tokens（缓存命中 1元/百万 Tokens）
    - 输出: 16元/百万 Tokens
    官方价格文档：https://api-docs.deepseek.com/zh-cn/quick_start/pricing/
    """
    stats = completion.usage
    
    # 尝试获取字段（兼容其他平台）
    hit = getattr(stats, 'prompt_cache_hit_tokens', 0)
    miss = getattr(stats, 'prompt_cache_miss_tokens', 
                  stats.prompt_tokens - hit if hasattr(stats, 'prompt_tokens') else 0)
    
    print(f"===== TOKEN 消耗明细 =====")
    # 仅在存在缓存机制时显示细节
    if hit + miss > 0:
        print(f"输入: {stats.prompt_tokens} tokens [缓存命中: {hit} | 未命中: {miss}]")
    else:
        print(f"输入: {stats.prompt_tokens} tokens")
    
    print(f"输出: {stats.completion_tokens} tokens")

    # 尝试获取推理过程详情
    details = getattr(stats, 'completion_tokens_details', None)
    reasoning = 0
    if details:
        if not isinstance(details, dict):
            details = getattr(details, 'dict', lambda: {})()
        # 尝试获取 reasoning_tokens
        reasoning = details.get('reasoning_tokens', 0)
        
        # 仅在存在推理tokens数量字段时处理
        if reasoning > 0:
            final = stats.completion_tokens - reasoning
            print(f"├─ 推理过程: {reasoning} tokens")
            print(f"└─ 最终回答: {final} tokens")
    
    print(f"总消耗: {stats.total_tokens} tokens")
    
    # 动态成本计算
    input_cost_total = (hit * cache_hit_cost + miss * input_cost) / 1_000_000
    output_cost_total = stats.completion_tokens * output_cost / 1_000_000
    total_cost = input_cost_total + output_cost_total
    
    print(f"\n===== 成本明细 =====")
    print(f"输入成本: ￥{input_cost_total:.4f} 元")
    print(f"输出成本: ￥{output_cost_total:.4f} 元")
    print(f"预估总成本: ￥{total_cost:.4f} 元")

print_reasoner_usage(completion)

===== TOKEN 消耗明细 =====
输入: 13 tokens [缓存命中: 0 | 未命中: 13]
输出: 147 tokens
总消耗: 160 tokens

===== 成本明细 =====
输入成本: ￥0.0001 元
输出成本: ￥0.0024 元
预估总成本: ￥0.0024 元


# 阿里云百炼

## 认识输出

### DeepSeek-Chat

In [17]:
from openai import OpenAI
import os

client = OpenAI(
    api_key="your-api-key", # 1：替换成对应的 API_Key，可以使用环境变量而非明文填写，即 api_key=os.getenv("SILICONFLOW_API_KEY")
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1", # 2：每个平台的 base_url 不同
)

# 单轮对话示例
completion = client.chat.completions.create(
    model="deepseek-v3", # 3：模型标识（model_id）可能存在差异
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant.'},
        {'role': 'user', 'content': '你是谁？'}
    ]
)

In [18]:
from pprint import pprint

pprint(completion.model_dump())

{'choices': [{'finish_reason': 'stop',
              'index': 0,
              'logprobs': None,
              'message': {'content': '我是DeepSeek '
                                     'Chat，一个由深度求索公司开发的智能助手，旨在通过自然语言处理和机器学习技术来提供信息查询、对话交流和解答问题等服务。如果您有任何问题或需要帮助，请随时告诉我！',
                          'function_call': None,
                          'refusal': None,
                          'role': 'assistant',
                          'tool_calls': None}}],
 'created': 1739067920,
 'id': 'chatcmpl-35b95e65-9196-926e-9bd4-d2198ce3fc06',
 'model': 'deepseek-v3',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': None,
 'usage': {'completion_tokens': 45, 'prompt_tokens': 12, 'total_tokens': 57}}


#### 获取模型回复（choices）

In [19]:
print(completion.choices[0].message.content)

我是DeepSeek Chat，一个由深度求索公司开发的智能助手，旨在通过自然语言处理和机器学习技术来提供信息查询、对话交流和解答问题等服务。如果您有任何问题或需要帮助，请随时告诉我！


#### 获取用量信息（usage）

In [20]:
def print_chat_usage(completion, input_cost=2.0, output_cost=8.0, cache_hit_cost=0.5):
    """
    参数：
    - input_cost: 输入价格（元/百万 Tokens）
    - output_cost: 输出价格（元/百万 Tokens）
    - cache_hit_cost: 缓存命中价格（当平台不支持时自动退化到全价模式）

    按 DeepSeek 聊天模型定价设定默认成本（单位：元）：
    - 输入: 2元/百万 Tokens（缓存命中 0.5元/百万 Tokens）
	- 输出: 8元/百万 Tokens
    官方价格文档：https://api-docs.deepseek.com/zh-cn/quick_start/pricing/
    """
    stats = completion.usage

    # 尝试获取字段（兼容其他平台）
    hit = getattr(stats, 'prompt_cache_hit_tokens', 0)
    miss = getattr(stats, 'prompt_cache_miss_tokens', 
                  stats.prompt_tokens - hit if hasattr(stats, 'prompt_tokens') else 0)

    print(f"===== TOKEN 消耗明细 =====")
    # 仅在存在缓存机制时显示细节
    if hit + miss > 0:
        print(f"输入: {stats.prompt_tokens} tokens [缓存命中: {hit} | 未命中: {miss}]")
    else:
        print(f"输入: {stats.prompt_tokens} tokens")

    print(f"输出: {stats.completion_tokens} tokens")
    print(f"总消耗: {stats.total_tokens} tokens")

    # 动态成本计算
    input_cost = (hit * cache_hit_cost + miss * input_cost) / 1_000_000
    output_cost = stats.completion_tokens * output_cost / 1_000_000
    total_cost = input_cost + output_cost

    print(f"\n===== 成本明细 =====")
    print(f"输入成本: ￥{input_cost:.4f} 元")
    print(f"输出成本: ￥{output_cost:.4f} 元")
    print(f"预估总成本: ￥{total_cost:.4f} 元")

print_chat_usage(completion)

===== TOKEN 消耗明细 =====
输入: 12 tokens [缓存命中: 0 | 未命中: 12]
输出: 45 tokens
总消耗: 57 tokens

===== 成本明细 =====
输入成本: ￥0.0000 元
输出成本: ￥0.0004 元
预估总成本: ￥0.0004 元


### DeepSeek-Reasoner

> 阿里云百炼的 deepseek-r1 API 可能会出现预期外的输出：重复/没有思维链或不解析 `<think>` 标签。

In [21]:
# 单轮对话示例
completion = client.chat.completions.create(
    model="deepseek-r1", # 3：换成推理模型
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant.'},
        {'role': 'user', 'content': '你是谁？'}
    ]
)

In [22]:
from pprint import pprint

pprint(completion.model_dump())

{'choices': [{'finish_reason': 'stop',
              'index': 0,
              'logprobs': None,
              'message': {'content': '您好！我是由中国的深度求索（DeepSeek）公司开发的智能助手DeepSeek-R1。有关模型和产品的详细内容请参考官方文档。',
                          'function_call': None,
                          'reasoning_content': '',
                          'refusal': None,
                          'role': 'assistant',
                          'tool_calls': None}}],
 'created': 1739067923,
 'id': 'chatcmpl-3e65620f-0cd6-94ff-9b85-90beb6929042',
 'model': 'deepseek-r1',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': None,
 'usage': {'completion_tokens': 38, 'prompt_tokens': 12, 'total_tokens': 50}}


#### 获取模型回复（choices）

In [23]:
import re

def parse_reasoner_response(completion):
    """
    参数：
    - completion (object): API 返回的对象
    
    返回：
    - (reasoning_content, reply_content)
    
    处理两种平台格式：
    1. 有独立 reasoning_content 字段的平台：DeepSeek 官方，硅基流动，百度智能云...
    2. 可能需要从 content 解析 <think> 标签的平台：阿里云百炼（偶尔会没有 reasoning_content）...
    """
    message = completion.choices[0].message
    
    # 尝试直接获取 reasoning_content 字段
    reasoning = getattr(message, 'reasoning_content', None)
    
    # 有 reasoning_content 时直接获取最终回复
    if reasoning:
        final_content = getattr(message, 'content', '')
    else:
        # 如果没有，则尝试从 content 解析
        content = getattr(message, 'content', '')
        
        # 使用非贪婪模式匹配 <think> 标签
        reasoning_match = re.search(
            r'<think>(.*?)</think>', 
            content, 
            re.DOTALL  # 允许跨行匹配
        )
        
        if reasoning_match:
            reasoning = reasoning_match.group(1).strip()
            # 从原始内容移除推理部分
            final_content = re.sub(
                r'<think>.*?</think>', 
                '', 
                content, 
                flags=re.DOTALL
            ).strip()
        else:
            reasoning = ''
            final_content = content
    
    return reasoning, final_content

reasoning_content, content = parse_reasoner_response(completion)

print(f"===== 模型推理过程 =====\n{reasoning_content}")
print(f"\n===== 最终回复 =====\n{content}")

===== 模型推理过程 =====


===== 最终回复 =====
您好！我是由中国的深度求索（DeepSeek）公司开发的智能助手DeepSeek-R1。有关模型和产品的详细内容请参考官方文档。


#### 获取用量信息（usage）

In [24]:
def print_reasoner_usage(completion, input_cost=4.0, output_cost=16.0, cache_hit_cost=1.0):
    """
    参数：
    - input_cost: 输入价格（元/百万 Tokens）
    - output_cost: 输出价格（元/百万 Tokens）
    - cache_hit_cost: 缓存命中价格（当平台不支持时自动退化到全价模式）

    按 DeepSeek 推理模型定价设定默认成本（单位：元）：
    - 输入: 4元/百万 Tokens（缓存命中 1元/百万 Tokens）
    - 输出: 16元/百万 Tokens
    官方价格文档：https://api-docs.deepseek.com/zh-cn/quick_start/pricing/
    """
    stats = completion.usage
    
    # 尝试获取字段（兼容其他平台）
    hit = getattr(stats, 'prompt_cache_hit_tokens', 0)
    miss = getattr(stats, 'prompt_cache_miss_tokens', 
                  stats.prompt_tokens - hit if hasattr(stats, 'prompt_tokens') else 0)
    
    print(f"===== TOKEN 消耗明细 =====")
    # 仅在存在缓存机制时显示细节
    if hit + miss > 0:
        print(f"输入: {stats.prompt_tokens} tokens [缓存命中: {hit} | 未命中: {miss}]")
    else:
        print(f"输入: {stats.prompt_tokens} tokens")
    
    print(f"输出: {stats.completion_tokens} tokens")

    # 尝试获取推理过程详情
    details = getattr(stats, 'completion_tokens_details', None)
    reasoning = 0
    if details:
        if not isinstance(details, dict):
            details = getattr(details, 'dict', lambda: {})()
        # 尝试获取 reasoning_tokens
        reasoning = details.get('reasoning_tokens', 0)
        
        # 仅在存在推理tokens数量字段时处理
        if reasoning > 0:
            final = stats.completion_tokens - reasoning
            print(f"├─ 推理过程: {reasoning} tokens")
            print(f"└─ 最终回答: {final} tokens")
    
    print(f"总消耗: {stats.total_tokens} tokens")
    
    # 动态成本计算
    input_cost_total = (hit * cache_hit_cost + miss * input_cost) / 1_000_000
    output_cost_total = stats.completion_tokens * output_cost / 1_000_000
    total_cost = input_cost_total + output_cost_total
    
    print(f"\n===== 成本明细 =====")
    print(f"输入成本: ￥{input_cost_total:.4f} 元")
    print(f"输出成本: ￥{output_cost_total:.4f} 元")
    print(f"预估总成本: ￥{total_cost:.4f} 元")

print_reasoner_usage(completion)

===== TOKEN 消耗明细 =====
输入: 12 tokens [缓存命中: 0 | 未命中: 12]
输出: 38 tokens
总消耗: 50 tokens

===== 成本明细 =====
输入成本: ￥0.0000 元
输出成本: ￥0.0006 元
预估总成本: ￥0.0007 元


# 百度智能云

## 认识输出

### DeepSeek-Chat

In [25]:
from openai import OpenAI
import os

client = OpenAI(
    api_key="your-api-key", # 1：替换成对应的 API_Key，可以使用环境变量而非明文填写，即 api_key=os.getenv("SILICONFLOW_API_KEY")
    base_url="https://qianfan.baidubce.com/v2" # 2：每个平台的 base_url 不同
)

# 单轮对话示例
completion = client.chat.completions.create(
    model="deepseek-v3", # 3：模型标识（model_id）可能存在差异
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant.'},
        {'role': 'user', 'content': '你是谁？'}
    ]
)

In [26]:
from pprint import pprint

pprint(completion.model_dump())

{'choices': [{'finish_reason': 'normal',
              'flag': 0,
              'index': 0,
              'logprobs': None,
              'message': {'content': '我是一个人工智能助手，由OpenAI开发，旨在通过自然语言处理和机器学习技术来回答问题、提供信息和协助用户完成各种任务。你可以问我任何问题，我会尽力提供帮助！',
                          'function_call': None,
                          'refusal': None,
                          'role': 'assistant',
                          'tool_calls': None}}],
 'created': 1739067928,
 'id': 'as-ivpqpffkd1',
 'model': 'deepseek-v3',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': None,
 'usage': {'completion_tokens': 36, 'prompt_tokens': 8, 'total_tokens': 44}}


#### 获取模型回复（choices）

In [27]:
print(completion.choices[0].message.content)

我是一个人工智能助手，由OpenAI开发，旨在通过自然语言处理和机器学习技术来回答问题、提供信息和协助用户完成各种任务。你可以问我任何问题，我会尽力提供帮助！


#### 获取用量信息（usage）

In [28]:
def print_chat_usage(completion, input_cost=2.0, output_cost=8.0, cache_hit_cost=0.5):
    """
    参数：
    - input_cost: 输入价格（元/百万 Tokens）
    - output_cost: 输出价格（元/百万 Tokens）
    - cache_hit_cost: 缓存命中价格（当平台不支持时自动退化到全价模式）

    按 DeepSeek 聊天模型定价设定默认成本（单位：元）：
    - 输入: 2元/百万 Tokens（缓存命中 0.5元/百万 Tokens）
	- 输出: 8元/百万 Tokens
    官方价格文档：https://api-docs.deepseek.com/zh-cn/quick_start/pricing/
    """
    stats = completion.usage

    # 尝试获取字段（兼容其他平台）
    hit = getattr(stats, 'prompt_cache_hit_tokens', 0)
    miss = getattr(stats, 'prompt_cache_miss_tokens', 
                  stats.prompt_tokens - hit if hasattr(stats, 'prompt_tokens') else 0)

    print(f"===== TOKEN 消耗明细 =====")
    # 仅在存在缓存机制时显示细节
    if hit + miss > 0:
        print(f"输入: {stats.prompt_tokens} tokens [缓存命中: {hit} | 未命中: {miss}]")
    else:
        print(f"输入: {stats.prompt_tokens} tokens")

    print(f"输出: {stats.completion_tokens} tokens")
    print(f"总消耗: {stats.total_tokens} tokens")

    # 动态成本计算
    input_cost = (hit * cache_hit_cost + miss * input_cost) / 1_000_000
    output_cost = stats.completion_tokens * output_cost / 1_000_000
    total_cost = input_cost + output_cost

    print(f"\n===== 成本明细 =====")
    print(f"输入成本: ￥{input_cost:.4f} 元")
    print(f"输出成本: ￥{output_cost:.4f} 元")
    print(f"预估总成本: ￥{total_cost:.4f} 元")

print_chat_usage(completion)

===== TOKEN 消耗明细 =====
输入: 8 tokens [缓存命中: 0 | 未命中: 8]
输出: 36 tokens
总消耗: 44 tokens

===== 成本明细 =====
输入成本: ￥0.0000 元
输出成本: ￥0.0003 元
预估总成本: ￥0.0003 元


### DeepSeek-Reasoner

In [29]:
# 单轮对话示例
completion = client.chat.completions.create(
    model="deepseek-r1", # 3：换成推理模型
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant.'},
        {'role': 'user', 'content': '你是谁？'}
    ]
)

In [30]:
from pprint import pprint

pprint(completion.model_dump())

{'choices': [{'finish_reason': 'normal',
              'flag': 0,
              'index': 0,
              'logprobs': None,
              'message': {'content': '\n'
                                     '\n'
                                     '您好！我是由中国的深度求索（DeepSeek）公司开发的智能助手DeepSeek-R1。如您有任何任何问题，我会尽我所能为您提供帮助。',
                          'function_call': None,
                          'reasoning_content': '嗯，用户问“你是谁？”，这是一个常见的问题，可能他们想了解我的基本信息或者功能。首先，我需要保持回答简洁明了，但也要全面。用户可能是初次使用，所以应该详细介绍我的功能和用途，避免使用技术术语，让回答更易懂。同时，要强调我的限制，比如数据截止到2023年，不能联网等，这样可以管理用户的预期。另外，要保持友好和专业的语气，让用户感觉被重视。可能用户后续会有更具体的问题，所以最后可以邀请他们提出更多需求，促进互动。需要确保回答结构清晰，分点说明，但不要太机械，自然流畅最好。\n',
                          'refusal': None,
                          'role': 'assistant',
                          'tool_calls': None}}],
 'created': 1739067946,
 'id': 'as-bxdt1ar9hb',
 'model': 'deepseek-r1',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': None,
 'usage': {'completion_tokens': 165,
           'com

#### 获取模型回复（choices）

In [31]:
# 获取推理思考过程（Reasoner特有字段）
reasoning_content = completion.choices[0].message.reasoning_content
print(f"===== 模型推理过程 =====\n{reasoning_content}")

# 获取模型回复内容（与之前相同）
content = completion.choices[0].message.content
print(f"===== 模型回复 =====\n{content}")

===== 模型推理过程 =====
嗯，用户问“你是谁？”，这是一个常见的问题，可能他们想了解我的基本信息或者功能。首先，我需要保持回答简洁明了，但也要全面。用户可能是初次使用，所以应该详细介绍我的功能和用途，避免使用技术术语，让回答更易懂。同时，要强调我的限制，比如数据截止到2023年，不能联网等，这样可以管理用户的预期。另外，要保持友好和专业的语气，让用户感觉被重视。可能用户后续会有更具体的问题，所以最后可以邀请他们提出更多需求，促进互动。需要确保回答结构清晰，分点说明，但不要太机械，自然流畅最好。

===== 模型回复 =====


您好！我是由中国的深度求索（DeepSeek）公司开发的智能助手DeepSeek-R1。如您有任何任何问题，我会尽我所能为您提供帮助。


#### 获取用量信息（usage）

In [32]:
def print_reasoner_usage(completion, input_cost=4.0, output_cost=16.0, cache_hit_cost=1.0):
    """
    参数：
    - input_cost: 输入价格（元/百万 Tokens）
    - output_cost: 输出价格（元/百万 Tokens）
    - cache_hit_cost: 缓存命中价格（当平台不支持时自动退化到全价模式）

    按 DeepSeek 推理模型定价设定默认成本（单位：元）：
    - 输入: 4元/百万 Tokens（缓存命中 1元/百万 Tokens）
    - 输出: 16元/百万 Tokens
    官方价格文档：https://api-docs.deepseek.com/zh-cn/quick_start/pricing/
    """
    stats = completion.usage
    
    # 尝试获取字段（兼容其他平台）
    hit = getattr(stats, 'prompt_cache_hit_tokens', 0)
    miss = getattr(stats, 'prompt_cache_miss_tokens', 
                  stats.prompt_tokens - hit if hasattr(stats, 'prompt_tokens') else 0)
    
    print(f"===== TOKEN 消耗明细 =====")
    # 仅在存在缓存机制时显示细节
    if hit + miss > 0:
        print(f"输入: {stats.prompt_tokens} tokens [缓存命中: {hit} | 未命中: {miss}]")
    else:
        print(f"输入: {stats.prompt_tokens} tokens")
    
    print(f"输出: {stats.completion_tokens} tokens")

    # 尝试获取推理过程详情
    details = getattr(stats, 'completion_tokens_details', None)
    reasoning = 0
    if details:
        if not isinstance(details, dict):
            details = getattr(details, 'dict', lambda: {})()
        # 尝试获取 reasoning_tokens
        reasoning = details.get('reasoning_tokens', 0)
        
        # 仅在存在推理tokens数量字段时处理
        if reasoning > 0:
            final = stats.completion_tokens - reasoning
            print(f"├─ 推理过程: {reasoning} tokens")
            print(f"└─ 最终回答: {final} tokens")
    
    print(f"总消耗: {stats.total_tokens} tokens")
    
    # 动态成本计算
    input_cost_total = (hit * cache_hit_cost + miss * input_cost) / 1_000_000
    output_cost_total = stats.completion_tokens * output_cost / 1_000_000
    total_cost = input_cost_total + output_cost_total
    
    print(f"\n===== 成本明细 =====")
    print(f"输入成本: ￥{input_cost_total:.4f} 元")
    print(f"输出成本: ￥{output_cost_total:.4f} 元")
    print(f"预估总成本: ￥{total_cost:.4f} 元")

print_reasoner_usage(completion)

===== TOKEN 消耗明细 =====
输入: 8 tokens [缓存命中: 0 | 未命中: 8]
输出: 165 tokens
├─ 推理过程: 131 tokens
└─ 最终回答: 34 tokens
总消耗: 173 tokens

===== 成本明细 =====
输入成本: ￥0.0000 元
输出成本: ￥0.0026 元
预估总成本: ￥0.0027 元


# 字节火山引擎

## 认识输出

### DeepSeek-Chat

In [33]:
from openai import OpenAI
import os

client = OpenAI(
    api_key="your-api-key", # 1：替换成对应的 API_Key，可以使用环境变量而非明文填写，即 api_key=os.getenv("SILICONFLOW_API_KEY")
    base_url="https://ark.cn-beijing.volces.com/api/v3" # 2：每个平台的 base_url 不同
)

# 单轮对话示例
completion = client.chat.completions.create(
    model="deepseek-v3-241226", # 3：模型标识（model_id）可能存在差异
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant.'},
        {'role': 'user', 'content': '你是谁？'}
    ]
)

In [34]:
from pprint import pprint

pprint(completion.model_dump())

{'choices': [{'finish_reason': 'stop',
              'index': 0,
              'logprobs': None,
              'message': {'content': '我是DeepSeek '
                                     'Chat，一个由深度求索公司开发的智能助手，旨在通过自然语言处理和机器学习技术来提供信息查询、对话交流和解答问题等服务。如有任何疑问或需要帮助，请随时告诉我。',
                          'function_call': None,
                          'refusal': None,
                          'role': 'assistant',
                          'tool_calls': None}}],
 'created': 1739067948,
 'id': '0217390679463949666a785a9e88d5b0179ae9e4e1f0138c10560',
 'model': 'deepseek-v3-241226',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': None,
 'usage': {'completion_tokens': 45,
           'completion_tokens_details': {'reasoning_tokens': 0},
           'prompt_tokens': 11,
           'prompt_tokens_details': {'cached_tokens': 0},
           'total_tokens': 56}}


#### 获取模型回复（choices）

In [35]:
print(completion.choices[0].message.content)

我是DeepSeek Chat，一个由深度求索公司开发的智能助手，旨在通过自然语言处理和机器学习技术来提供信息查询、对话交流和解答问题等服务。如有任何疑问或需要帮助，请随时告诉我。


#### 获取用量信息（usage）

In [36]:
def print_chat_usage(completion, input_cost=2.0, output_cost=8.0, cache_hit_cost=0.5):
    """
    参数：
    - input_cost: 输入价格（元/百万 Tokens）
    - output_cost: 输出价格（元/百万 Tokens）
    - cache_hit_cost: 缓存命中价格（当平台不支持时自动退化到全价模式）

    按 DeepSeek 聊天模型定价设定默认成本（单位：元）：
    - 输入: 2元/百万 Tokens（缓存命中 0.5元/百万 Tokens）
	- 输出: 8元/百万 Tokens
    官方价格文档：https://api-docs.deepseek.com/zh-cn/quick_start/pricing/
    """
    stats = completion.usage

    # 尝试获取字段（兼容其他平台）
    hit = getattr(stats, 'prompt_cache_hit_tokens', 0)
    miss = getattr(stats, 'prompt_cache_miss_tokens', 
                  stats.prompt_tokens - hit if hasattr(stats, 'prompt_tokens') else 0)

    print(f"===== TOKEN 消耗明细 =====")
    # 仅在存在缓存机制时显示细节
    if hit + miss > 0:
        print(f"输入: {stats.prompt_tokens} tokens [缓存命中: {hit} | 未命中: {miss}]")
    else:
        print(f"输入: {stats.prompt_tokens} tokens")

    print(f"输出: {stats.completion_tokens} tokens")
    print(f"总消耗: {stats.total_tokens} tokens")

    # 动态成本计算
    input_cost = (hit * cache_hit_cost + miss * input_cost) / 1_000_000
    output_cost = stats.completion_tokens * output_cost / 1_000_000
    total_cost = input_cost + output_cost

    print(f"\n===== 成本明细 =====")
    print(f"输入成本: ￥{input_cost:.4f} 元")
    print(f"输出成本: ￥{output_cost:.4f} 元")
    print(f"预估总成本: ￥{total_cost:.4f} 元")

print_chat_usage(completion)

===== TOKEN 消耗明细 =====
输入: 11 tokens [缓存命中: 0 | 未命中: 11]
输出: 45 tokens
总消耗: 56 tokens

===== 成本明细 =====
输入成本: ￥0.0000 元
输出成本: ￥0.0004 元
预估总成本: ￥0.0004 元


### DeepSeek-Reasoner

In [37]:
# 单轮对话示例
completion = client.chat.completions.create(
    model="deepseek-r1-250120", # 3：换成推理模型
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant.'},
        {'role': 'user', 'content': '你是谁？'}
    ]
)

In [38]:
from pprint import pprint

pprint(completion.model_dump())

{'choices': [{'finish_reason': 'stop',
              'index': 0,
              'logprobs': None,
              'message': {'content': '\n'
                                     '\n'
                                     '您好！我是由中国的深度求索（DeepSeek）公司开发的智能助手DeepSeek-R1。如您有任何任何问题，我会尽我所能为您提供帮助。',
                          'function_call': None,
                          'reasoning_content': '好的，用户问我“你是谁？”，我需要用中文回答。首先，我应该介绍自己的身份，说明我是由DeepSeek开发的智能助手。接着，可以提到我的主要功能是提供信息、解答问题和协助完成任务。要保持回答简洁明了，符合用户的需求。可能用户想知道我的开发背景或者用途，所以需要涵盖这些方面。同时，要确保语气友好，让用户感到被帮助。然后，要检查有没有遗漏的重要信息，比如是否需要提到具体的服务范围或技术特点。最后，用自然的口语化表达，避免使用过于正式或复杂的句子结构，让用户更容易理解。\n',
                          'refusal': None,
                          'role': 'assistant',
                          'tool_calls': None}}],
 'created': 1739067957,
 'id': '0217390679487739666a785a9e88d5b0179ae9e4e1f0138eb752d',
 'model': 'deepseek-r1-250120',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': None,
 'usage': {'completion_tokens': 163

#### 获取模型回复（choices）

In [39]:
# 获取推理思考过程（Reasoner特有字段）
reasoning_content = completion.choices[0].message.reasoning_content
print(f"===== 模型推理过程 =====\n{reasoning_content}")

# 获取模型回复内容（与之前相同）
content = completion.choices[0].message.content
print(f"===== 模型回复 =====\n{content}")

===== 模型推理过程 =====
好的，用户问我“你是谁？”，我需要用中文回答。首先，我应该介绍自己的身份，说明我是由DeepSeek开发的智能助手。接着，可以提到我的主要功能是提供信息、解答问题和协助完成任务。要保持回答简洁明了，符合用户的需求。可能用户想知道我的开发背景或者用途，所以需要涵盖这些方面。同时，要确保语气友好，让用户感到被帮助。然后，要检查有没有遗漏的重要信息，比如是否需要提到具体的服务范围或技术特点。最后，用自然的口语化表达，避免使用过于正式或复杂的句子结构，让用户更容易理解。

===== 模型回复 =====


您好！我是由中国的深度求索（DeepSeek）公司开发的智能助手DeepSeek-R1。如您有任何任何问题，我会尽我所能为您提供帮助。


#### 获取用量信息（usage）

In [40]:
def print_reasoner_usage(completion, input_cost=4.0, output_cost=16.0, cache_hit_cost=1.0):
    """
    参数：
    - input_cost: 输入价格（元/百万 Tokens）
    - output_cost: 输出价格（元/百万 Tokens）
    - cache_hit_cost: 缓存命中价格（当平台不支持时自动退化到全价模式）

    按 DeepSeek 推理模型定价设定默认成本（单位：元）：
    - 输入: 4元/百万 Tokens（缓存命中 1元/百万 Tokens）
    - 输出: 16元/百万 Tokens
    官方价格文档：https://api-docs.deepseek.com/zh-cn/quick_start/pricing/
    """
    stats = completion.usage
    
    # 尝试获取字段（兼容其他平台）
    hit = getattr(stats, 'prompt_cache_hit_tokens', 0)
    miss = getattr(stats, 'prompt_cache_miss_tokens', 
                  stats.prompt_tokens - hit if hasattr(stats, 'prompt_tokens') else 0)
    
    print(f"===== TOKEN 消耗明细 =====")
    # 仅在存在缓存机制时显示细节
    if hit + miss > 0:
        print(f"输入: {stats.prompt_tokens} tokens [缓存命中: {hit} | 未命中: {miss}]")
    else:
        print(f"输入: {stats.prompt_tokens} tokens")
    
    print(f"输出: {stats.completion_tokens} tokens")

    # 尝试获取推理过程详情
    details = getattr(stats, 'completion_tokens_details', None)
    reasoning = 0
    if details:
        if not isinstance(details, dict):
            details = getattr(details, 'dict', lambda: {})()
        # 尝试获取 reasoning_tokens
        reasoning = details.get('reasoning_tokens', 0)
        
        # 仅在存在推理tokens数量字段时处理
        if reasoning > 0:
            final = stats.completion_tokens - reasoning
            print(f"├─ 推理过程: {reasoning} tokens")
            print(f"└─ 最终回答: {final} tokens")
    
    print(f"总消耗: {stats.total_tokens} tokens")
    
    # 动态成本计算
    input_cost_total = (hit * cache_hit_cost + miss * input_cost) / 1_000_000
    output_cost_total = stats.completion_tokens * output_cost / 1_000_000
    total_cost = input_cost_total + output_cost_total
    
    print(f"\n===== 成本明细 =====")
    print(f"输入成本: ￥{input_cost_total:.4f} 元")
    print(f"输出成本: ￥{output_cost_total:.4f} 元")
    print(f"预估总成本: ￥{total_cost:.4f} 元")

print_reasoner_usage(completion)

===== TOKEN 消耗明细 =====
输入: 13 tokens [缓存命中: 0 | 未命中: 13]
输出: 163 tokens
├─ 推理过程: 125 tokens
└─ 最终回答: 38 tokens
总消耗: 176 tokens

===== 成本明细 =====
输入成本: ￥0.0001 元
输出成本: ￥0.0026 元
预估总成本: ￥0.0027 元
